In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

 # 경고 메시지를 제어하는 모듈
import warnings 
warnings.filterwarnings("ignore", category=RuntimeWarning) # RuntimeWarning 경고 메시지를 무시하도록 설정

bike_df = pd.read_csv('./bike_train.csv')
print(bike_df.shape)
bike_df.head()

In [ ]:
bike_df.info()

In [ ]:
# 문자열을 datetime 타입으로 변경 (예제의 데이터 세트의 datetime칼럼명과 동일, 둘이 혼동 주의)
# 문자열을 datetime 타입으로 변환하는 apply(pd.to_datetime)
bike_df['datetime']=bike_df.datetime.apply(pd.to_datetime)

# datetime 타입에서 년, 월, 일, 시간 추출
bike_df['year'] = bike_df.datetime.apply(lambda x: x.year)
bike_df['month'] = bike_df.datetime.apply(lambda x: x.month)
bike_df['day'] = bike_df.datetime.apply(lambda x: x.day)
bike_df['hour'] = bike_df.datetime.apply(lambda x: x.hour)
bike_df.head(3)

In [ ]:
# casual= 사전에 등록X 대여 횟수, registered= 사전에 등록0 대여 횟수, casual+registered=count
# casual,registered 따로 필요X 오히려 상관도 높아 예측 저해 우려
drop_columns = ['datetime', 'casual', 'registered']
bike_df.drop(drop_columns, axis=1, inplace=True) # axis=1 열 방향, inplace=True 데이터프레임 직접 변경하여 새로 반환하지X

In [ ]:
fig, axs=plt.subplots(figsize=(16,8), ncols=4, nrows=2)
cat_features = ['year','month','season','weather','day','hour','holiday','workingday']

# cat_features에 있는 모든 칼럼별로 개별 칼럼값에 따른 count의 합을 barplot으로 시각화
for i, feature in enumerate(cat_features) :
    row = int(i/4)
    col = i%4
    # 사본의 barplot을 이용해 칼럼값에 따른 count의 합을 표현
    sns.barplot(x=feature, y='count', data=bike_df, ax=axs[row][col])

# season 봄(1), 여름(2), 가을(3), 겨울(4)
# holiday 1=토,일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
# workingday 1=토,일 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
# weather 1(맑음,약간 구름 낀 흐림), 2(안개,안개+흐림), 3(가벼운 눈, 가벼운 비+천둥), 4(심한 눈/비, 천둥/번개)

In [ ]:
# RMSLE, MSE, RMSE 수행 함수 만들기

from sklearn.metrics import mean_squared_error, mean_absolute_error

# log값 변환 시 NaN등의 이슈로 log()가 아닌 log1p()를 이용해 RMSLE계산
def rmsle(y,pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

# 사이킷런의 mean_square_error()를 이용해 RMSE 계산
def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

# MSE, RMSE, RMSLE를 모두 계산
def evaluate_regr(y,pred):
    rmsle_val = rmsle(y,pred)
    rmse_val = rmse(y,pred)
    # MAE는 사이킷런의 mean_absolute_error()로 계산
    mae_val = mean_absolute_error(y,pred)
    print('RMSLE; {0:.3f}, RMSE: {1:.3F}, MAE: {2:.3F}'.format(rmsle_val, rmse_val, mae_val))

In [ ]:
# LinearRegression 객체를 이용한 회귀 예측

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso

y_target = bike_df['count']
X_features = bike_df.drop(['count'],axis=1,inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

evaluate_regr(y_test,pred)

In [ ]:
# 실제값과 예측값 차이 확인
def get_top_error_data(y_test,pred,n_tops=5):
    # DataFrame의 칼럼으로 실제 대여 횟수(count)와 예측값을 서로 비교할 수 있도록 생성
    result_df = pd.DataFrame(y_test.values, columns=['real_count'])
    result_df['predicted_count']=np.round(pred)
    result_df['diff'] = np.abs(result_df['real_count'] - result_df['predicted_count'])

# 예측값과 실제값이 가장 큰 데이터 순으로 출력
    print(result_df.sort_values('diff',ascending=False)[:n_tops])

get_top_error_data(y_test, pred, n_tops=5)

In [ ]:
# Target값인 count 칼럼이 정규분포를 이루는지 확인
y_target.hist()

In [ ]:
# log1p()이용해 정규 분포 형태로 변환
y_log_transform = np.log1p(y_target)
y_log_transform.hist()

In [ ]:
# 타깃 칼럼인 count 값을 log1p로 로그 변환
y_target_log = np.log1p(y_target)

# 로그 변환된 y_target_log를 반영해 학습/테스트 데이터 세트 분할
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target_log, test_size=0.3, random_state=0)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

# 테스트 데이터 세트의 Target값은 로그 변환됐으므로 다시 expm1을 이용해 원래 스케일로 변환
y_test_exp = np.expm1(y_test)

# 예측값 역시 로그 변환된 타깃 기반으로 학습돼 예측됐으므로 다시 expm1로 스케일 변환
pred_exp = np.expm1(pred)

evaluate_regr(y_test_exp, pred_exp)

In [ ]:
# 개별 피처들 인코팅
# 각 피처의 회귀 계숫값 시각화
coef = pd.Series(lr_reg.coef_, index=X_features.columns)
coef_sort = coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
# 카테고리 값을 선형 회귀에 사용할 경우 회귀 계수를 연산할 때 이 숫자형 값에 크게 영향을 받음

# 'year', 'month', 'day', 'hour'등의 피처들을 One Hot Encoding
X_features_ohe = pd.get_dummies(X_features, columns=['year','month','day','hour','holiday','workingday','season','weather'])

In [ ]:
# 원-핫 인코딩이 적용된 피처 데이터 세트 기반으로 학습/예측 데이터 분할.
X_train, X_test, y_train, y_test = train_test_split(X_features_ohe, y_target_log, test_size=0.3, random_state=0)

# 모델과 학습/테스트 데이터 세트를 입력하면 성능 평가 수치를 반환
def get_model_predict(model, X_train, X_test, y_train, y_test, is_expm1=False):
    model.fit(X_train, y_train) # 모델을 학습 데이터로 학습
    pred=model.predict(X_test) # 테스트 데이터로 예측 수행
    if is_expm1:  # log 변환을 되돌릴 필요가 있는 경우
        y_test = np.expm1(y_test) # y_test를 로그 변환 되돌림
        pred = np.expm1(pred) # 예측 결과도 로그 변환 되돌림
    print('###',model.__class__.__name__,'###')
    evaluate_regr(y_test, pred) # 모델의 성능 평가 수행
# end of function get_model_predict

# 모델별로 평가 수행
lr_reg = LinearRegression() # 선형 회귀 모델
ridge_reg = Ridge(alpha=10) # 릿지 회귀 모델 (정규화 강도 alpha=10)
lasso_reg = Lasso(alpha=0.01) # 라쏘 회귀 모델 (정규화 강도 alpha=0.01)

# 각 모델별로 성능 평가 수행
for model in [lr_reg, ridge_reg, lasso_reg]:
    get_model_predict(model,X_train,X_test,y_train,y_test,is_expm1=True)

In [ ]:
# 회귀 계수 상위 20개 피처 추출
coef = pd.Series(lr_reg.coef_, index=X_features_ohe.columns)
coef_sort = coef.sort_values(ascending=False)[:20]
sns.barplot(x=coef_sort.values , y=coef_sort.index)

In [ ]:
# 회귀 트리를 이용해 회귀 예측 수행
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 랜덤 포레스트, GBM, XGBoost, LightGBM model별로 평가 수행
rf_reg = RandomForestRegressor(n_estimators=500)
gbm_reg = GradientBoostingRegressor(n_estimators=500)
xgb_reg = XGBRegressor(n_estimators=500)
lgbm_reg = LGBMRegressor(n_estimators=500)

for model in [rf_reg, gbm_reg, xgb_reg, lgbm_reg]:
    # XGBoost의 경우 DataFrame이 입력될 경우 버전에 따라 오류 발생 가능. ndarray로 변환.
    get_model_predict(model,X_train.values, X_test.values, y_train.values, y_test.values, is_expm1=True)